In [3]:
from collections import Counter
import numpy as np
import os
import pandas as pd
import spacy
import spacy.attrs
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from gensim.models.word2vec import Word2Vec
import gensim.downloader as api
import re

C:\Users\enire\anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [ ]:
pd = pd.read_csv("C19_measures.csv") #for extracting # of COVID words analysis

In [ ]:
text = pd.read_csv("C19_for_embeddings.csv")  #for extracting text

In [ ]:
text

In [ ]:
pd1 = pd[(pd["covid_words"]>0) | (pd["subreddit"].str.contains("covid", case=False)) | (pd["subreddit"].str.contains("vaccine", case=False))]
pd1 = pd1[pd1["afterCOVID"]==0]
pd1.to_csv("beforeCOVIDdata.csv", index = False)

pd2= pd[(pd["covid_words"]>0) | (pd["subreddit"].str.contains("covid", case=False)) | (pd["subreddit"].str.contains("vaccine", case=False))]
pd2 = pd2[pd2["afterCOVID"]==1]
pd2.to_csv("afterCOVIDdata.csv", index = False)

In [ ]:
before_filter = pd.DataFrame()   #creating new data frame for beforecovid posts w/ text & save in csv
for id in pd1.index:
    print(id)
    df3 = {"User_Name": text.iloc[id,0],"created_utc": text.iloc[id,1],"afterCOVID": 0, "subreddit": pd1.loc[id,"subreddit"],"covid_words":pd1.loc[id,"covid_words"] ,"sentiment":pd1.loc[id,"sentiment"],"text":text.iloc[id,3]}
    before_filter = before_filter.append(df3, ignore_index = True)
    
before_filter.to_csv("before_filter.csv", index = False)
before_filter = pd.read_csv("before_filter.csv")

In [ ]:
after_filter = pd.DataFrame()  #creating new data frame for aftercovid posts w/ text & save in csv
for id in pd2.index:
    print(id)
    df3 = {"User_Name": text.iloc[id,0],"created_utc": text.iloc[id,1],"afterCOVID": 1, "subreddit": pd2.loc[id,"subreddit"],"covid_words":pd2.loc[id,"covid_words"] ,"sentiment":pd2.loc[id,"sentiment"],"text":text.iloc[id,3]}
    after_filter = after_filter.append(df3, ignore_index = True)
    
after_filter.to_csv("after_filter.csv", index = False)
after_filter = pd.read_csv("after_filter.csv")
print("done")

In [4]:
#reading the csv 
before_filter = pd.read_csv("before_filter.csv")  
after_filter = pd.read_csv("after_filter.csv")

In [5]:
# sampling
before_filter_sample = before_filter[1:5000]
after_filter_sample = after_filter[1:5000]

In [ ]:
after_filter_sample

In [6]:
#load spacey nlp
verbose = False
nlp = spacy.load('en_core_web_md')
nlp.max_length = 2000000

In [7]:
#filtering purposes
punctuation = [".", "!", "?"]

In [8]:
#remove emoji & amp
def remove_emoji(string):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\U0001f919"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               u"\u200b"
                               u"\u2010"
                               u"\u03b1"
                               u"\u2070-\u2260"
                               u"\u0131"
                               u"\u011f-\u015f"
                               u"\u202f"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', string)

In [30]:
before_text = ""     
column = before_filter_sample.text
for each in column:
    if each[-1] not in punctuation:
        each = each +"."
    before_text = before_text + " " + each


In [31]:
after_text = ""
column = after_filter_sample.text
for each in column:
    if each[-1] not in punctuation:
        each = each +"."
    after_text = after_text + " " + each


In [36]:
before_text = remove_emoji(before_text.lower())
after_text = remove_emoji(after_text.lower())

In [37]:
#save purposes 
with open("before_text.txt", "w") as f:
    f.write(before_text)
    
with open("after_text.txt","w" ) as f:
    f.write(after_text)

In [9]:
#open saved file
with open("before_text.txt", "r") as f:
    before_text1 = f.read()

with open("after_text.txt", "r") as f:
    after_text1 = f.read()

Creating the w2vdoc models 

In [27]:
w2vdoc = nlp(before_text1)

In [28]:
sentlist = [sent.text.strip(". !?") for sent in w2vdoc.sents]

In [11]:
w2vdoc2 = nlp(after_text1)

In [12]:
sentlist = [sent.text.strip(". !?") for sent in w2vdoc2.sents]

In [29]:
"""Algorithm to create a list of tokenized sentences"""
listolists = []
for each in sentlist:
    thissent = []
    hold = nlp(each)
    for eachtoken in hold:
        if eachtoken.is_space or eachtoken.is_stop:
            continue
        else:
            eachtoken = eachtoken.lemma_  #convert to lemma form bc we have a small data set
            eachtoken = str(eachtoken)
            thissent.append(eachtoken)
    if thissent != []:
        listolists.append(thissent)

In [30]:
"""Run word2vec to create a w2v model"""
model = Word2Vec(listolists, min_count=2)
model.wv.save_word2vec_format('BinaryLandbackModel.bin')

In [15]:
model2 = Word2Vec(listolists, min_count=2)
model2.wv.save_word2vec_format('BinaryLandbackModel2.bin')

In [22]:
#model = Word2Vec.load('BinaryLandbackModel.bin')
#model2 = Word2Vec.load('BinaryLandBackModel2.bin')

In [31]:
def GetCosSimList(model, word, verbose:bool=True):
    """Returns and can print a list of the words in our doc that are most
    cosine-similar to a word the function is passed"""
    word = word.lower()
    simlist = model.wv.similar_by_vector(word, topn=25, restrict_vocab=None)
    if verbose:
        print("Top 25 words similar to '" + word + "':")
        ct = 1
        for tup in simlist:
            print("(" + str(ct) + "):  " + tup[0] + "------" + str(tup[1]))
            ct += 1
    return simlist

print("Before COVID:")
GetCosSimList(model, "covid")

print("After COVID:")
GetCosSimList(model2, "covid")


Before COVID:
Top 25 words similar to 'covid':
(1):  doctor------0.9951851963996887
(2):  baby------0.9941032528877258
(3):  tell------0.9940159916877747
(4):  need------0.9925158619880676
(5):  true------0.9923438429832458
(6):  yeah------0.9922130107879639
(7):  know------0.9915797710418701
(8):  mean------0.9913991689682007
(9):  accord------0.9913396239280701
(10):  ward------0.9906923174858093
(11):  number------0.9906181693077087
(12):  not------0.9904426336288452
(13):  plant------0.9902167916297913
(14):  guess------0.9901776909828186
(15):  case------0.9901561737060547
(16):  consider------0.9901284575462341
(17):  right------0.990083634853363
(18):  responsible------0.9898399710655212
(19):  flu------0.9894385933876038
(20):  precaution------0.9893040657043457
(21):  delay------0.9892234206199646
(22):  guy------0.9891847968101501
(23):  scenario------0.9891186952590942
(24):  google------0.9888888001441956
(25):  immunity------0.9887418150901794
After COVID:
Top 25 words sim

[('contagious', 0.9962073564529419),
 ('confirm', 0.9961245059967041),
 ('expose', 0.9956574440002441),
 ('case', 0.9955943822860718),
 ('asymptomatic', 0.9955438375473022),
 ('infection', 0.9952675700187683),
 ('result', 0.9951648116111755),
 ('got', 0.995132327079773),
 ('load', 0.9951081275939941),
 ('infectious', 0.9950907230377197),
 ('20', 0.9948573708534241),
 ('isolate', 0.9947651624679565),
 ('accurate', 0.9947428107261658),
 ('long', 0.9946513175964355),
 ('january', 0.9945401549339294),
 ('free', 0.9944214820861816),
 ('icu', 0.9943399429321289),
 ('likely', 0.9942951798439026),
 ('clinic', 0.9942656755447388),
 ('vax', 0.994174599647522),
 ('begin', 0.9941478967666626),
 ('testing', 0.9941408634185791),
 ('1st', 0.9940490126609802),
 ('longhaul', 0.9940169453620911),
 ('nasal', 0.9939851760864258)]